# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import time
import re

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

import pickle

pd.set_option('display.max_columns', 100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# load data from database
engine = create_engine('sqlite:///disasterDB.db')
df = pd.read_sql_table("Responses", con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# computes a summary of statistics pertaining to the DataFrame columns
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
# remove child_alone column because it has only zeros
df = df.drop(['child_alone'],axis=1)

In [12]:
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    """
    Tokenization function: to tokenize a given text 
    
    Arguments:
        text: Text message which needs to be tokenized
    Output:
        clean_tokens: List of tokens extracted from the provided text
    """
    
    # use regular expression to detect all urls in the provided text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # replace url with a urlplaceholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, 'urlplaceholder')

    # extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
     # remove stopwords
    tokens_stopwords_removed = [w for w in tokens if w not in stopwords.words("english")]
    
    # lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # list of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens_stopwords_removed]
    return clean_tokens

#### Test the output of Tokenization function

In [14]:
# test Tokenization function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'haiti', ',', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
# train the model
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
# classification report
def display_report(x_actual, y_actual, model):
    """
    display_report function: to calculate the evaluation metrics for a model
    
    Arguments:
        x_actual: X_test or X_train of the model
        y_actual: y_test or y_train of the model
        model: the model 
    """
    
    # get the predicted value
    y_pred = model.predict(x_actual)
    
    for i, col_name in enumerate(y_actual.columns):               
        print('______________________________________________________\n')
        print('Column: {}\n'.format(col_name))
        print(classification_report(y_actual.iloc[:,i], y_pred[:,i]))          

In [19]:
# predict on the test data
display_report(X_test, y_test, pipeline)

______________________________________________________

Column: related

             precision    recall  f1-score   support

          0       0.64      0.41      0.50      1554
          1       0.83      0.93      0.87      4953
          2       0.30      0.19      0.23        47

avg / total       0.78      0.80      0.78      6554

______________________________________________________

Column: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5434
          1       0.82      0.43      0.56      1120

avg / total       0.88      0.89      0.87      6554

______________________________________________________

Column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

______________________________________________________

Column: aid_related

        

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
# predict on the train data
display_report(X_train, y_train, pipeline)

______________________________________________________

Column: related

             precision    recall  f1-score   support

          0       0.99      0.98      0.98      4568
          1       0.99      1.00      0.99     14953
          2       0.97      0.85      0.91       141

avg / total       0.99      0.99      0.99     19662

______________________________________________________

Column: request

             precision    recall  f1-score   support

          0       0.98      1.00      0.99     16308
          1       1.00      0.92      0.96      3354

avg / total       0.99      0.99      0.99     19662

______________________________________________________

Column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19571
          1       1.00      0.74      0.85        91

avg / total       1.00      1.00      1.00     19662

______________________________________________________

Column: aid_related

        

             precision    recall  f1-score   support

          0       0.98      1.00      0.99     15877
          1       1.00      0.91      0.95      3785

avg / total       0.98      0.98      0.98     19662



### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
def build_1stTunedModel():
    """
    build_1stTunedModel function: to improve the model 

    Output:
        cv: model with GridSearch
    """
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier( RandomForestClassifier() ))
    ])
    
    parameters = {'tfidf__use_idf' : [True, False],        
              'vect__max_df' : [1.0, 0.8],              
              'clf__estimator__min_samples_split' : [2, 4],
              'clf__estimator__n_estimators' : [10, 20]
    }
    
    cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1)
    
    return cv

In [23]:
# train first tuned model
begin = time.time()

firstTuned = build_1stTunedModel()
firstTuned.fit(X_train, y_train)

finish = time.time()
print("Model Train Time = ", finish - begin, " Seconds")

Model Train Time =  2782.7540390491486  Seconds


In [25]:
firstTuned.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
# predict on the train data for 1st improved model
display_report(X_train, y_train, firstTuned)

______________________________________________________

Column: related

             precision    recall  f1-score   support

          0       1.00      0.98      0.99      4568
          1       0.99      1.00      1.00     14953
          2       0.99      0.94      0.97       141

avg / total       1.00      1.00      1.00     19662

______________________________________________________

Column: request

             precision    recall  f1-score   support

          0       0.99      1.00      1.00     16308
          1       1.00      0.97      0.99      3354

avg / total       1.00      1.00      1.00     19662

______________________________________________________

Column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19571
          1       1.00      0.82      0.90        91

avg / total       1.00      1.00      1.00     19662

______________________________________________________

Column: aid_related

        

             precision    recall  f1-score   support

          0       0.99      1.00      1.00     15877
          1       1.00      0.96      0.98      3785

avg / total       0.99      0.99      0.99     19662



In [26]:
# predict on the test data for 1st improved model
display_report(X_test, y_test, firstTuned)

______________________________________________________

Column: related

             precision    recall  f1-score   support

          0       0.68      0.39      0.49      1554
          1       0.82      0.94      0.88      4953
          2       0.43      0.06      0.11        47

avg / total       0.79      0.81      0.78      6554

______________________________________________________

Column: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.94      5434
          1       0.84      0.43      0.57      1120

avg / total       0.88      0.89      0.87      6554

______________________________________________________

Column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

______________________________________________________

Column: aid_related

        

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
def build_2ndTunedModel():
    """
    build_2ndTunedModel function: to improve the model 

    Output:
        cv: model with GridSearch
    """
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(SVC()))
    ])

    parameters = { 
        'features__text_pipeline__vect__max_df': [1.0],
        'features__text_pipeline__tfidf__use_idf': [True],
        'clf__estimator__kernel': ['poly'],
        'clf__estimator__degree': [1, 2, 3]        
    }

    cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1)
    
    return cv

In [37]:
# train second tuned model
begin = time.time()

secondTuned = build_2ndTunedModel()
secondTuned.fit(X_train, y_train)

finish = time.time()
print("Model Train Time = ", finish - begin, " Seconds")

Model Train Time =  5393.317162513733  Seconds


In [39]:
secondTuned.best_params_

{'clf__estimator__degree': 1,
 'clf__estimator__kernel': 'poly',
 'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_df': 1.0}

In [40]:
# predict on the train data for 2nd improved model
display_report(X_train, y_train, secondTuned)

______________________________________________________

Column: related

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      4568
          1       0.76      1.00      0.86     14953
          2       0.00      0.00      0.00       141

avg / total       0.58      0.76      0.66     19662

______________________________________________________

Column: request

             precision    recall  f1-score   support

          0       0.83      1.00      0.91     16308
          1       0.00      0.00      0.00      3354

avg / total       0.69      0.83      0.75     19662

______________________________________________________

Column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19571
          1       0.00      0.00      0.00        91

avg / total       0.99      1.00      0.99     19662

______________________________________________________

Column: aid_related

        

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
# predict on the test data for 2nd improved model
display_report(X_test, y_test, secondTuned)

______________________________________________________

Column: related

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1554
          1       0.76      1.00      0.86      4953
          2       0.00      0.00      0.00        47

avg / total       0.57      0.76      0.65      6554

______________________________________________________

Column: request

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5434
          1       0.00      0.00      0.00      1120

avg / total       0.69      0.83      0.75      6554

______________________________________________________

Column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

______________________________________________________

Column: aid_related

        

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### We could noticed that the second tuning of our model didn't give us a better results. Therefore, we should keep using the first tuned model because it was giving more accurate results for all categories.

### 9. Export your model as a pickle file

In [ ]:
my_pickle = open('disasterReponse.sav', 'wb')
pickle.dump(firstTuned, my_pickle)
my_pickle.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.